In [1]:
import sagemaker
from sagemaker.pytorch import PyTorch

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [3]:
estimator = PyTorch(
    entry_point='train.py',
    role=role,
    framework_version='1.8.0',
    py_version='py3',
    instance_count=1,
    instance_type='ml.m5.xlarge',  
    hyperparameters={
        'epochs': 10,
        'batch-size': 32,
    },
    metric_definitions=[
        {'Name': 'train:loss', 'Regex': 'Loss: ([0-9\\.]+)'}
    ]
)


In [ ]:
estimator.fit({'training': 's3://sagemaker-eu-north-1-495599732227/preprocessed2'})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2024-10-16-09-47-01-213


2024-10-16 09:47:02 Starting - Starting the training job...
2024-10-16 09:47:34 Starting - Preparing the instances for training...
2024-10-16 09:47:56 Downloading - Downloading input data...
2024-10-16 09:48:31 Downloading - Downloading the training image...
2024-10-16 09:48:57 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-10-16 09:49:07,045 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-10-16 09:49:07,047 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-10-16 09:49:07,057 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-10-16 09:49:07,060 sagemaker_pytorch_container.training INFO     Invoking user training script.
2024-10-16 09:49:07,216 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed

In [8]:
model_data = 's3://sagemaker-eu-north-1-495599732227/pytorch-training-2024-10-16-09-47-01-213/output/model.tar.gz'  # Replace with your S3 URI
pytorch_model = PyTorchModel(
    model_data=model_data,
    role=role,
    framework_version='1.8.0',  
    py_version='py3',  
    entry_point='inference.py'
)

predictor = pytorch_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.xlarge'  
)

INFO:sagemaker:Repacking model artifact (s3://sagemaker-eu-north-1-495599732227/pytorch-training-2024-10-16-09-47-01-213/output/model.tar.gz), script artifact (None), and dependencies ([]) into single tar.gz file located at s3://sagemaker-eu-north-1-495599732227/pytorch-inference-2024-10-16-18-15-16-651/model.tar.gz. This may take some time depending on model size...
INFO:sagemaker:Creating model with name: pytorch-inference-2024-10-16-18-15-26-102
INFO:sagemaker:Creating endpoint-config with name pytorch-inference-2024-10-16-18-15-26-836
INFO:sagemaker:Creating endpoint with name pytorch-inference-2024-10-16-18-15-26-836


------!